# Tasks
This notebook contains the solutions for the Computational Theory tasks

# Task 1: Binary Representations

In this task, we implement four functions based on the FIPS 180-4 Secure Hash Standard:
- `rotl(x, n=1)`: Rotate bits of a 32-bit unsigned integer to the left by n positions.
- `rotr(x, n=1)`: Rotate bits of a 32-bit unsigned integer to the right by n positions.
- `ch(x, y, z)`: Choose bits from `y` when the corresponding bit in `x` is 1; otherwise, choose from `z`.
- `maj(x, y, z)`: Compute the majority vote for each bit position (i.e., output a 1 if at least two of the three inputs have a 1 in that bit).

**Reference:** FIPS 180-4, Secure Hash Standard (SHS)

In [153]:
# Define the function to rotate left (rotl) for a 32-bit unsigned integer.
def rotl(x, n=1):
    """
    Rotate the 32-bit unsigned integer x to the left by n bits.
    
    Parameters:
        x (int): 32-bit unsigned integer.
        n (int): Number of positions to rotate (default is 1).
    
    Returns:
        int: The result of rotating x to the left by n positions.
    """
    # Shift left by n, shift right by (32 - n), and combine with bitwise OR.
    # Use bitwise AND with 0xFFFFFFFF to ensure the result stays within 32 bits.
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Define the function to rotate right (rotr) for a 32-bit unsigned integer.
def rotr(x, n=1):
    """
    Rotate the 32-bit unsigned integer x to the right by n bits.
    
    Parameters:
        x (int): 32-bit unsigned integer.
        n (int): Number of positions to rotate (default is 1).
    
    Returns:
        int: The result of rotating x to the right by n positions.
    """
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# Define the choice function (ch) that selects bits from y or z based on x.
def ch(x, y, z):
    """
    Choose bits from y or z based on the bits in x.
    
    For each bit position, if x has a 1, take the corresponding bit from y;
    otherwise, take the bit from z.
    
    Parameters:
        x, y, z (int): 32-bit unsigned integers.
    
    Returns:
        int: The result after selecting bits.
    """
    # (x AND y) gives bits from y where x is 1.
    # (~x AND z) gives bits from z where x is 0.
    return (x & y) ^ ((~x) & z)  # XOR here works like OR for non-overlapping bits

# Define the majority function (maj) which outputs the majority vote of bits.
def maj(x, y, z):
    """
    Compute the majority of the bits in x, y, and z.
    
    For each bit position, the output bit is 1 if at least two of the inputs
    have a 1 in that position.
    
    Parameters:
        x, y, z (int): 32-bit unsigned integers.
    
    Returns:
        int: The result of the majority vote.
    """
    # Using the formula: (x AND y) OR (x AND z) OR (y AND z)
    return (x & y) | (x & z) | (y & z)

## Testing the Functions

In the following cell, we will demonstrate example usages and simple tests for each of the functions:
- We'll provide sample inputs to check that our functions perform the expected bit rotations.
- We will test `ch` and `maj` to confirm they return the correct values based on binary logic.
- Each example is accompanied by comments to explain what is being tested.


In [154]:
# Test examples for the functions

# Example for rotl:
x = 0x12345678  # Example 32-bit number in hexadecimal
rotated_left = rotl(x, 4)
print("rotl(0x12345678, 4):", hex(rotated_left))
# Expected: The bits should be rotated 4 positions to the left

# Example for rotr:
rotated_right = rotr(x, 4)
print("rotr(0x12345678, 4):", hex(rotated_right))
# Expected: The bits should be rotated 4 positions to the right

# Example for ch:
# Let's choose arbitrary 32-bit numbers for testing.
a = 0b10101010101010101010101010101010  # Pattern: alternating bits
b = 0xFFFFFFFF  # All bits 1
c = 0x00000000  # All bits 0
chosen = ch(a, b, c)
print("ch(a, b, c):", bin(chosen))
# Expected: For each bit position, if bit in a is 1, bit from b (1) is chosen; else, bit from c (0)

# Example for maj:
# Using three arbitrary numbers.
x_val = 0b11001100110011001100110011001100
y_val = 0b10101010101010101010101010101010
z_val = 0b11110000111100001111000011110000
majority = maj(x_val, y_val, z_val)
print("maj(x, y, z):", bin(majority))
# Expected: For each bit position, the output is 1 if at least two of x, y, and z have a 1.


rotl(0x12345678, 4): 0x23456781
rotr(0x12345678, 4): 0x81234567
ch(a, b, c): 0b10101010101010101010101010101010
maj(x, y, z): 0b11101000111010001110100011101000


# Task 2: Hash Functions

In this task, we convert a hash function from C (from *The C Programming Language* by Kernighan and Ritchie) into Python. The original C code is:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


In [155]:
def custom_hash(s: str) -> int:
    """
    Compute a hash value for the string s using a method similar to the C version.
    
    For each character in the string, the hash value is computed as:
        hashval = ord(character) + 31 * hashval
    and finally, the hash value is reduced modulo 101.
    
    Parameters:
        s (str): The input string.
        
    Returns:
        int: The computed hash value in the range [0, 100].
    """
    hashval = 0
    for ch in s:
        hashval = ord(ch) + 31 * hashval
    return hashval % 101

In [156]:
# Test examples for the custom_hash function

test_strings = ["hello", "world", "hash", "function", "Python", ""]

for s in test_strings:
    print(f"Hash for '{s}': {custom_hash(s)}")

Hash for 'hello': 17
Hash for 'world': 34
Hash for 'hash': 15
Hash for 'function': 100
Hash for 'Python': 81
Hash for '': 0


# Task 3: SHA256 Padding

In this task, we write a Python function that calculates the SHA256 padding for a given file. The function takes a file path as input and prints (in hexadecimal) the padding that would be applied to that file.

According to the specification, the following is appended to a message:

- A single 1 bit (represented as 0x80 when appended as a byte)
- Enough 0 bits so that the total length (in bits) of the padded message is the smallest multiple of 512 that is greater than or equal to the original length plus 1 bit plus 64 bits for the length field
- The original message length (in bits) encoded as a big-endian 64-bit unsigned integer

For example, for a file containing the three bytes "abc" (which is 24 bits), the output should be:

```
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18
```


In [157]:
def sha256_padding(file_path: str) -> bytes:
    """
    Calculate the SHA256 padding for the contents of a given file.
    
    The padding is constructed as follows:
      1. Append a single '1' bit (0x80 in byte form).
      2. Append enough '0' bits (as 0x00 bytes) so that the total length (in bytes)
         of (message + padding, not including the length field) is congruent to 56 mod 64.
      3. Append the original message length (in bits) as an 8-byte (64-bit) big-endian integer.
      
    Parameters:
        file_path (str): The path to the input file.
        
    Returns:
        bytes: The padding bytes that would be appended.
    """
    # Read the file contents as bytes
    with open(file_path, "rb") as f:
        data = f.read()
    
    original_length_bits = len(data) * 8  # Message length in bits
    # Append the '1' bit as the byte 0x80 (10000000 in binary)
    padding = b'\x80'
    
    # Calculate the number of padding bytes needed to reach 56 mod 64
    # (56 bytes = 448 bits, leaving 8 bytes for the length field to reach a multiple of 64 bytes)
    current_length = len(data) + 1  # +1 for the 0x80 byte
    # Compute the number of pad bytes needed: if current_length mod 64 > 56, we need to wrap around.
    pad_length = (56 - (current_length % 64)) % 64
    # Append pad_length zero bytes
    padding += b'\x00' * pad_length
    
    # Append the 64-bit big-endian length field
    padding += original_length_bits.to_bytes(8, byteorder='big')
    
    return padding

def print_padding_hex(padding: bytes) -> None:
    """
    Print the padding bytes in hex format, with a space between each byte.
    
    Parameters:
        padding (bytes): The padding bytes to print.
    """
    # Format each byte as a two-digit hex number and join them with spaces.
    hex_output = ' '.join(f"{b:02x}" for b in padding)
    print(hex_output)

In [158]:
# For testing, let's create a temporary file containing "abc" (three bytes)
import tempfile

# Create a temporary file with the content "abc"
with tempfile.NamedTemporaryFile(delete=False) as tmp:
    tmp.write(b'abc')
    tmp_path = tmp.name

# Calculate the padding for the temporary file
padding_bytes = sha256_padding(tmp_path)

# Print the padding in hex format
print_padding_hex(padding_bytes)

# Optionally, remove the temporary file after testing
import os
os.remove(tmp_path)


80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


## Explanation of the SHA256 Padding Process

The SHA256 padding is calculated in three steps:

1. **Append a 1 bit:**  
   This is represented as the byte `0x80` (binary `10000000`).

2. **Append 0 bits:**  
   We then append as many zero bytes (`0x00`) as needed so that the total length of the message 
   (including the `0x80` byte but excluding the length field) becomes congruent to 56 modulo 64.  
   This ensures that, after appending an 8-byte (64-bit) representation of the original message length, 
   the total length is a multiple of 64 bytes (512 bits).

3. **Append the 64-bit length field:**  
   Finally, the original message length (in bits) is appended as an 8-byte big-endian integer.

For the example with the file containing "abc" (which is 24 bits long):
- After the message, the `0x80` byte is appended.
- Then, enough zero bytes are added so that the total length (excluding the length field) is 56 bytes.
- Finally, the 8-byte representation of 24 (0x18 in hex) is appended.


# Task 4: Prime Numbers

In this task, we calculate the first 100 prime numbers using two different algorithms. We will implement:

1. **Algorithm 1:** A simple trial division method.
2. **Algorithm 2:** The Sieve of Eratosthenes.

Below, each algorithm is implemented in a separate code cell. We then test both methods and explain how they work.


In [159]:
def is_prime_trial(n: int) -> bool:
    """
    Check if n is a prime number using trial division.
    
    This function tests divisibility of n by all integers from 2 up to the square root of n.
    If any divisor is found, n is not prime.
    
    Parameters:
        n (int): The number to test.
        
    Returns:
        bool: True if n is prime, False otherwise.
    """
    if n < 2:
        return False
    # Check for factors from 2 to int(sqrt(n))
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def first_100_primes_trial() -> list:
    """
    Generate the first 100 prime numbers using trial division.
    
    Returns:
        list: A list of the first 100 prime numbers.
    """
    primes = []
    candidate = 2
    while len(primes) < 100:
        if is_prime_trial(candidate):
            primes.append(candidate)
        candidate += 1
    return primes

# Calculate and display the first 100 primes using trial division
primes_trial = first_100_primes_trial()
print("First 100 primes (Trial Division):")
print(primes_trial)


First 100 primes (Trial Division):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


In [160]:
def sieve_of_eratosthenes(limit: int) -> list:
    """
    Generate a list of prime numbers up to a given limit using the Sieve of Eratosthenes.
    
    Parameters:
        limit (int): The upper bound to generate primes (inclusive).
        
    Returns:
        list: A list of prime numbers up to the given limit.
    """
    sieve = [True] * (limit + 1)
    sieve[0:2] = [False, False]  # 0 and 1 are not prime
    p = 2
    while p * p <= limit:
        if sieve[p]:
            # Mark multiples of p as non-prime
            for i in range(p * p, limit + 1, p):
                sieve[i] = False
        p += 1
    return [i for i, is_prime in enumerate(sieve) if is_prime]

def first_100_primes_sieve() -> list:
    """
    Generate the first 100 prime numbers using the Sieve of Eratosthenes.
    
    Since the exact upper limit is not known beforehand, we start with a guess and increase it if needed.
    
    Returns:
        list: A list of the first 100 prime numbers.
    """
    # Initial guess for an upper bound
    upper_bound = 600  
    while True:
        primes = sieve_of_eratosthenes(upper_bound)
        if len(primes) >= 100:
            return primes[:100]
        upper_bound *= 2  # Increase bound and try again

# Calculate and display the first 100 primes using the Sieve of Eratosthenes
primes_sieve = first_100_primes_sieve()
print("First 100 primes (Sieve of Eratosthenes):")
print(primes_sieve)


First 100 primes (Sieve of Eratosthenes):
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Explanation of the Algorithms

### Trial Division Method
- **How It Works:**  
  The trial division algorithm checks each candidate number by testing for divisibility by every integer from 2 up to the square root of the candidate. If no divisor is found, the number is prime.
- **Pros and Cons:**  
  - **Pros:** Simple to implement.
  - **Cons:** Inefficient for larger numbers because it performs many divisions.

### Sieve of Eratosthenes
- **How It Works:**  
  The sieve creates a boolean list (the "sieve") representing numbers up to a given limit. Starting from 2, it marks multiples of each prime as non-prime. The remaining numbers marked as prime are collected.
- **Pros and Cons:**  
  - **Pros:** Efficient for generating all primes up to a large limit.
  - **Cons:** Requires knowing or guessing an upper bound and uses extra memory for the sieve.


In [161]:
# Verify that both methods yield the same 100 primes
if primes_trial == primes_sieve:
    print("Both algorithms produced the same list of primes.")
else:
    print("There is a discrepancy between the two algorithms.")

# Optionally, print the list of primes
print("First 100 primes:")
print(primes_trial)


Both algorithms produced the same list of primes.
First 100 primes:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


# Task 5: Roots

In this task, we calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

For each prime number _p_, we perform the following steps:
1. Compute the square root: `sqrt(p)`.
2. Extract the fractional part: `sqrt(p) - floor(sqrt(p))`.
3. Multiply the fractional part by 2^32 (i.e. 4294967296) and take the integer part.
4. Represent this 32-bit value in hexadecimal.


In [162]:
import math

def first_100_primes() -> list:
    """
    Generate the first 100 prime numbers using a simple trial division method.
    
    Returns:
        list: A list containing the first 100 prime numbers.
    """
    primes = []
    candidate = 2
    while len(primes) < 100:
        is_prime = True
        for i in range(2, int(math.sqrt(candidate)) + 1):
            if candidate % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(candidate)
        candidate += 1
    return primes

# Generate and display the first 100 primes
primes = first_100_primes()
print("First 100 primes:")
print(primes)

First 100 primes:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


In [163]:
def fractional_32bits(prime: int) -> int:
    """
    Calculate the first 32 bits of the fractional part of the square root of a prime number.
    
    Steps:
      1. Compute sqrt(prime).
      2. Subtract floor(sqrt(prime)) to get the fractional part.
      3. Multiply the fractional part by 2^32 and take the integer part.
    
    Parameters:
        prime (int): A prime number.
        
    Returns:
        int: The 32-bit integer representing the fractional part of sqrt(prime).
    """
    sqrt_value = math.sqrt(prime)
    fractional_part = sqrt_value - math.floor(sqrt_value)
    value_32 = int(fractional_part * (2**32))
    return value_32

# Example: Calculate the 32-bit fractional part for a sample prime (e.g., 2)
print(f"32-bit fractional part for sqrt(2): {format(fractional_32bits(2), '08x')}")


32-bit fractional part for sqrt(2): 6a09e667


In [164]:
# Compute the 32-bit fractional part (in hexadecimal) for each of the first 100 primes
results = []
for p in primes:
    frac_value = fractional_32bits(p)
    hex_value = format(frac_value, '08x')  # Format as an 8-digit hexadecimal number
    results.append((p, hex_value))

# Print the results in a tabular format
print("Prime    32-bit Fractional Part (Hex)")
for prime, hex_val in results:
    print(f"{prime:<8} {hex_val}")


Prime    32-bit Fractional Part (Hex)
2        6a09e667
3        bb67ae85
5        3c6ef372
7        a54ff53a
11       510e527f
13       9b05688c
17       1f83d9ab
19       5be0cd19
23       cbbb9d5d
29       629a292a
31       9159015a
37       152fecd8
41       67332667
43       8eb44a87
47       db0c2e0d
53       47b5481d
59       ae5f9156
61       cf6c85d3
67       2f73477d
71       6d1826ca
73       8b43d457
79       e360b596
83       1c456002
89       6f196331
97       d94ebeb1
101      0cc4a611
103      261dc1f2
107      5815a7be
109      70b7ed67
113      a1513c69
127      44f93635
131      720dcdfd
137      b467369e
139      ca320b75
149      34e0d42e
151      49c7d9bd
157      87abb9f2
163      c463a2fc
167      ec3fc3f3
173      27277f6d
179      610bebf2
181      7420b49e
191      d1fd8a33
193      e4773594
197      092197f6
199      1b530c95
211      869d6342
223      eee52e4f
227      11076689
229      21fba37b
233      43ab9fb6
239      75a9f91d
241      86305019
251     

## Explanation of the Process

For each prime number _p_, the following steps are performed:

1. **Square Root Calculation:**  
   Compute `sqrt(p)` using Python's math library.

2. **Fractional Part Extraction:**  
   Subtract the integer part (`floor(sqrt(p))`) from `sqrt(p)` to obtain the fractional part.

3. **Conversion to 32 Bits:**  
   Multiply the fractional part by 2^32 (i.e. 4294967296). The integer part of this product is the first 32 bits of the fractional part.

4. **Hexadecimal Representation:**  
   The resulting integer is formatted as an 8-digit hexadecimal number, representing the 32-bit value.

This process is similar to the method used in SHA-256, where the first 32 bits of the fractional parts of the square roots of selected prime numbers are used as constants.


# Task 6: Proof of Work

In this task, we need to find the English word (or words) that yield a SHA256 hash with the greatest number of consecutive 0 bits at the beginning (i.e. the most leading zero bits).

We also need to include proof that any word we list is in at least one English dictionary. In our solution, we will load an English dictionary from the file `dolph_dictionary.txt` (which is in the same directory as our script) and search through its words.


In [165]:
import hashlib

def count_leading_zero_bits(hash_bytes: bytes) -> int:
    """
    Count the number of consecutive 0 bits from the beginning of the hash bytes.
    """
    count = 0
    for b in hash_bytes:
        for i in range(8):
            # Check the most significant bit first
            if b & (1 << (7 - i)):
                return count
            count += 1
    return count

# Load the dictionary from the local file "dolph_dictionary.txt"
try:
    with open("dolph_dictionary.txt", "r") as f:
        dictionary_words = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    # Fallback: use a small list if the file isn't found
    dictionary_words = ["example", "test", "prime", "zero", "one", "bitcoin", "crypto"]

print(f"Loaded {len(dictionary_words)} words from the dictionary (dolph_dictionary.txt).")


Loaded 172823 words from the dictionary (dolph_dictionary.txt).


In [166]:
max_zeros = -1
max_words = []

# Process each word from the dictionary
for word in dictionary_words:
    # Compute the SHA256 hash of the word (encoded in UTF-8)
    hash_bytes = hashlib.sha256(word.encode('utf-8')).digest()
    zeros = count_leading_zero_bits(hash_bytes)
    
    # Update our maximum if we find a word with more leading zeros
    if zeros > max_zeros:
        max_zeros = zeros
        max_words = [word]
    elif zeros == max_zeros:
        max_words.append(word)

print(f"Maximum number of leading zero bits found: {max_zeros}")
print("Word(s) with the maximum leading zero bits in their SHA256 hash:")
print(max_words)


Maximum number of leading zero bits found: 18
Word(s) with the maximum leading zero bits in their SHA256 hash:
['goaltenders']


In [167]:
# For each word found with the maximum number of leading zero bits,
# confirm that it exists in the dictionary (since we loaded them from dolph_dictionary.txt).
print("Proof of dictionary:")
for word in max_words:
    if word in dictionary_words:
        print(f"The word '{word}' is present in the English dictionary (dolph_dictionary.txt).")
    else:
        print(f"ERROR: '{word}' was not found in the dictionary!")


Proof of dictionary:
The word 'goaltenders' is present in the English dictionary (dolph_dictionary.txt).


## Explanation of the Approach

1. **SHA256 Hash and Leading Zero Bits:**
   - For each word, we compute its SHA256 hash using Python’s `hashlib` module.
   - We then count the number of consecutive 0 bits from the beginning of the hash digest.
   - This is done by iterating over each byte (starting with the most significant bit) and counting until the first 1 is encountered.

2. **Searching Through an English Dictionary:**
   - We load words from the local file `dolph_dictionary.txt`, which contains a list of English words.
   - We iterate over every word, compute its SHA256 hash, and track the word(s) with the maximum number of leading 0 bits.

3. **Proof of Dictionary Membership:**
   - We verify that each word with the maximum leading zeros is indeed present in `dolph_dictionary.txt`.
   - This serves as proof that the word is in at least one English dictionary.

By running these cells, you will determine which English word(s) yield the highest number of leading 0 bits in their SHA256 hash and verify their dictionary membership.
